In [ ]:
import os
import random
import numpy as np
import torchvision

from PIL import Image, ImageOps
from glob import glob
from torchvision import transforms

In [2]:
def random_subset_images(root_dir: str, percent: float, extensions: set = {'.jpg', '.jpeg', '.png', '.bmp'}) -> list[str]:
    """
    Randomly selects a subset of image file paths from the given root directory (recursively).
    
    Args:
        root_dir (str): The root directory containing images.
        percent (float): Fraction of images to select (0 < percent <= 1).
        extensions (set): Allowed image file extensions (lowercase).
    
    Returns:
        List[str]: Randomly selected image file paths.
    """
    # Gather all image file paths
    all_images = [y for x in os.walk(root_dir) 
                    for y in glob(os.path.join(x[0], '*')) 
                    if os.path.splitext(y)[1].lower() in extensions]
    
    if not (0 < percent <= 1):
        raise ValueError("percent must be between 0 and 1 (exclusive of 0)")
    
    num_to_select = int(len(all_images) * percent)
    selected = random.sample(all_images, num_to_select)

    return selected

In [ ]:
def augment(image_path: str):
     """
    Randomly selects a subset of image file paths from the given root directory (recursively).
    
    Args:
        image_path (str): The root directory containing images.
    """
    image = Image.open(image_path).convert('RGB')
    base_dir = os.path.dirname(image_path)
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    ext = os.path.splitext(image_path)[1]

    transform_list = [
        transforms.RandomApply([transforms.RandomRotation(degrees=30)], p=0.25),
        transforms.RandomApply([transforms.RandomHorizontalFlip(p=1.0)], p=0.25),
        transforms.RandomApply([transforms.RandomVerticalFlip(p=1.0)], p=0.25),
        transforms.RandomApply([transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.0))], p=0.05),
        transforms.RandomApply([transforms.RandomInvert(p=1.0)], p=0.025),
        transforms.RandomApply([transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)], p=0.025),
        transforms.RandomApply([transforms.RandomAffine(degrees=15, translate=(0.1, 0.1))], p=0.05),
        transforms.RandomApply([transforms.RandomAdjustSharpness(sharpness_factor=2)], p=0.05),
        transforms.RandomApply([transforms.RandomEqualize()], p=0.025)
    ]

    composed = transforms.Compose(transform_list)

    augmented = composed(image)

    save_path = os.path.join(base_dir, f"{base_name}_da{ext}")
    augmented.save(save_path)

In [ ]:
root_image_dir = "ROOT DIRECTORY"

for path in random_subset_images(root_dir=root_image_dir, percent=.4):
    augment(image_path=path)